## Import libraries

In [37]:
import numpy as np
import os
import seaborn as sn
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import pandas as pd
from google.colab import drive

Hardware specification

In [ ]:
def check_gpu():
    import tensorflow as tf
    print("tf.test.is_built_with_cuda()")
    print(tf.test.is_built_with_cuda())

    print()
    print("tf.config.list_physical_devices('GPU')")
    print(tf.config.list_physical_devices('GPU'))

    print()
    print("tf.config.experimental.list_physical_devices('GPU')")
    print(tf.config.experimental.list_physical_devices('GPU'))

check_gpu() # Google Colab doesn't have a GPU?

In [41]:
import tensorflow as tf

# Use CPU for training
#tf.config.set_visible_devices([], 'GPU')

# Use GPU for training (if available)
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU'), 'GPU')

# Check the device placement
print("Device:", tf.config.list_logical_devices())

Device: [LogicalDevice(name='/device:CPU:0', device_type='CPU')]


#### Google Colab Google Drive mounting

In [22]:
drive.mount('/content/drive')

# Set the path to the dataset folder
data_folder = '/content/drive/MyDrive/data'

files = os.listdir(data_folder)
for file in files:
    print(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
train
test
pred


## Data loading and data preparation

In [35]:
def get_image_info(image_path):
    image = cv2.imread(image_path)

    if image is not None:
        # Image shape
        height, width, channels = image.shape
        print("Image shape:", height, "x", width, "x", channels)

        # Image size in bytes
        image_size = os.path.getsize(image_path)
        print("Image size:", image_size, "bytes")

        # Image data type
        image_dtype = image.dtype
        print("Image data type:", image_dtype)

        # Image color space
        if channels == 1:
            color_space = "Grayscale"
        elif channels == 3:
            color_space = "RGB"
        else:
            color_space = "Unknown"
        print("Image color space:", color_space)

    else:
        print("Failed to read the image:", image_path)


get_image_info("/content/drive/MyDrive/data/train/buildings/10006.jpg")

Image shape: 150 x 150 x 3
Image size: 16802 bytes
Image data type: uint8
Image color space: RGB


In [30]:
from sklearn.utils import shuffle
from tqdm import tqdm

class_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]
class_names_label = {class_name: i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)

def count_images(folder_path):
    count = 0
    for _, _, files in os.walk(folder_path):
        count += len(files)
    return count

train_counts = [count_images(os.path.join(data_folder, 'train', class_name)) for class_name in class_names]
test_counts = [count_images(os.path.join(data_folder, 'test', class_name)) for class_name in class_names]
print("Train Images:", train_counts)
print("Test Images:", test_counts)

Train Images: [2191, 2271, 2404, 2512, 2274, 2382]
Test Images: [437, 477, 553, 525, 527, 501]


### Load images in batches into RAM

In [48]:
from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle

class My_Custom_Generator(Sequence):
    def __init__(self, folder_path, max_images, batch_size):
        self.folder_path = folder_path
        self.max_images = max_images
        self.batch_size = batch_size
        self.images, self.labels = self.load_images()

    def load_images(self):
        print("Loading " + self.folder_path)
        images = []
        labels = []
        images_per_folder = int(self.max_images / len(class_names))

        for folder in tqdm(os.listdir(self.folder_path)):
            num_images_loaded_this_folder = 0
            if folder not in class_names_label:
                continue
            label = class_names_label[folder]
            # Iterate through each image in this folder
            for file in os.listdir(os.path.join(self.folder_path, folder)):
                if num_images_loaded_this_folder >= images_per_folder:
                    break
                # Get the path name of the image
                img_path = os.path.join(os.path.join(self.folder_path, folder), file)
                # Open the image
                image = imread(img_path)
                image = resize(image, (150, 150, 3))
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                num_images_loaded_this_folder += 1

        print("Loaded " + self.folder_path)
        images, labels = shuffle(images, labels, random_state=3)
        images = np.array(images, dtype='float16') / 255.0
        labels = np.array(labels, dtype='int8')
        labels = tf.keras.utils.to_categorical(labels, num_classes=6)  # Convert labels to one-hot encoding
        return images, labels

    def __len__(self):
        return int(np.ceil(len(self.images) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_images = self.images[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]
        return batch_images, batch_labels

train_generator = My_Custom_Generator(os.path.join(data_folder, 'train'), max_images=7000, batch_size=64)
test_generator = My_Custom_Generator(os.path.join(data_folder, 'test'), max_images=1750, batch_size=64)
test_labels = test_generator.labels

n_train = train_generator.labels.shape[0]
n_test = test_generator.labels.shape[0]
test_percentage = (n_test / (n_train + n_test)) * 100

print("Train images:", n_train)
print("Test images:", n_test)
print("Test percentage: {}%".format(round(test_percentage)))

print("train_generator.images.shape:", train_generator.images.shape)
print("train_generator.labels[:10]:", train_generator.labels[:10])
print("test_generator.images.shape:", test_generator.images.shape)
print("test_generator.labels[:10]:", test_generator.labels[:10])

Loading /content/drive/MyDrive/data/train


100%|██████████| 6/6 [01:20<00:00, 13.50s/it]


Loaded /content/drive/MyDrive/data/train
Loading /content/drive/MyDrive/data/test


100%|██████████| 6/6 [00:19<00:00,  3.25s/it]


Loaded /content/drive/MyDrive/data/test
Train images: 6996
Test images: 1746
Test percentage: 20%
train_generator.images.shape: (6996, 150, 150, 3)
train_generator.labels[:10]: [[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
test_generator.images.shape: (1746, 150, 150, 3)
test_generator.labels[:10]: [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


## Multi-class Classification using Convulotinal Neural Networks (CNNs)

We are using a pre-trained ResNet50 model is used for multi-class image classification. The ResNet50 model is loaded without its top layer and its layers are frozen to prevent them from being trained. A new sequential model is created on top of the base model, consisting of a flatten layer, a dense layer with ReLU activation, and a dense layer with softmax activation for the output. The model is compiled with the Adam optimizer and categorical cross-entropy loss. The training and test images are preprocessed by scaling their values between 0 and 1. The model is then trained using the training images and labels, with a batch size of 32 and for 10 epochs. The validation data is provided using the test images and labels. After training, the model is saved for future use.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from sklearn.utils import shuffle

# Load the ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model's layers
base_model.trainable = False

# Create a new model on top of the base model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(6, activation='softmax'))  # Assuming 6 scene classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Rest of your code for data loading and generator initialization...

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

# Save the model
model.save('scene_classification_model.h5')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 256)               524544    
                                                                 
 dense_13 (Dense)            (None, 6)                 1542      
                                                                 
Total params: 24,113,798
Trainable params: 526,086
Non-trainable params: 23,587,712
_________________________________________________________________
Epoch 1/10
110/110 [==============================] - 606s 5s/step - loss: 1.8152 - accuracy: 0.1631 -

## Learning Curve and Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix

def plot_loss_accuracy(history):
    if not isinstance(history, dict):
        history = history.history

    num_epochs = len(history["accuracy"])
    x_values = range(1, num_epochs + 1)  # Generate x-axis values starting from 1

    # Plot accuracy
    plt.plot(x_values, history['accuracy'], label = "Train acc")
    plt.plot(x_values, history['val_accuracy'], label = "Validation acc")
    plt.title("Learning curve")
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()
    plt.xticks(range(0, num_epochs + 1, 2), range(0, num_epochs + 1, 2))
    plt.show()

    # Plot loss function
    plt.plot(x_values, history['loss'], label = "Train loss")
    plt.plot(x_values, history['val_loss'], label = "Validation loss")
    plt.title("Learning curve")
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.xticks(range(0, num_epochs + 1, 2), range(0, num_epochs + 1, 2))
    plt.show()

def plot_confusion_matrix(model, verbose=1):
    class_names = ["buildings", "forest", "glacier", "mountain", "sea", "street" ]
    preds = model.predict(test_generator, workers=8, verbose=verbose)
    preds_labels = np.argmax(preds, axis=1)
    cm = confusion_matrix(test_labels, preds_labels)
    ax = plt.axes()
    sn.heatmap(cm, annot=True, fmt="d",
               annot_kws={"size": 10},
               xticklabels=class_names,
               yticklabels=class_names, ax = ax)
    ax.set_title('Confusion matrix')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    plt.show()